PokeAPI data analysis on pokemon types and prefered attributes on early generations

at first we will set all the libraries and variables used along

In [1]:
import requests, json, pandas as pd, plotly.express as px

In [43]:
#base url for usage in functions
base_url = 'https://pokeapi.co/api/v2/'
#library of pokemon info
pokemon_info = {'Name': [], 'Types': [], 'Attribute': [], 'IV': [],}

Then, we will do functions to filter the pokemons we wish to use, and functions to gather the info about types and its IV's

In [3]:
#function to gather pokemon per gen(made to also get other gens)
def catch_pokemon_gen(n):
    url = f'https://pokeapi.co/api/v2/generation/{n}'
    p = requests.get(url)
    info = p.json()
    poke_list = []

    for pokemon in info['pokemon_species']:
        name = pokemon['name']
        poke_list.append(name)

    return poke_list

gen1 = catch_pokemon_gen(1)
gen2 = catch_pokemon_gen(2)
gen3 = catch_pokemon_gen(3)
gen4 = catch_pokemon_gen(4)

In [4]:
#function to gather pokemon types
def catch_pokemon_type(name):
    url = base_url + 'pokemon/' + name
    p = requests.get(url)
    info = p.json()
    types = [type['type']['name'] for type in info['types']]#since a pokemon can have 2 types, we read both and add them in a list
    return types

In [5]:
#function to gather pokemon info on its attributes
def catch_pokemon_iv(name):
    url = base_url + 'pokemon/' + name
    p = requests.get(url)
    info = p.json()
    attributes = ['hp', 'attack', 'defense', 'special-attack', 'special-defense', 'speed']
    highest_attribute = attributes[0]
    highest_attribute_iv = 0
    
    for i in range(len(attributes)):
        iv_value = info['stats'][i]['base_stat']

        if iv_value > highest_attribute_iv:
            highest_attribute_iv = iv_value
            highest_attribute = attributes[i]

    return highest_attribute, highest_attribute_iv

Now, using a function we shall aggregate all info we gather into the dictionary

In [6]:
#function for gathering info in a DF
def catch_all_info(list):
    for pokemon in list:
        try:
            types = catch_pokemon_type(pokemon)
            attribute, iv = catch_pokemon_iv(pokemon)
            pokemon_info['Name'].append(pokemon)
            pokemon_info['Types'].append(types)
            pokemon_info['Attribute'].append(attribute)
            pokemon_info['IV'].append(iv)
        except Exception as e:
            print(f'error on pokemon {pokemon}')
    return pokemon_info

In [40]:
gen1_info = catch_all_info(gen1)

In [41]:
gen2_info = catch_all_info(gen2)

In [9]:
gen3_info = catch_all_info(gen3)

error on pokemon deoxys


In [10]:
gen4_info = catch_all_info(gen4)

error on pokemon wormadam
error on pokemon giratina
error on pokemon shaymin


Here, we create dataframes for each generation we wish to use for our analysis

In [29]:
gen1df = pd.DataFrame(gen1_info)
gen1df.index += 1
gen2df = pd.DataFrame(gen2_info)
gen2df.index += 1
gen3df = pd.DataFrame(gen3_info)
gen3df.index += 1
gen4df = pd.DataFrame(gen4_info)
gen4df.index += 1

We then combine all pokemons into a single dataframe for a overall view

In [12]:
#sum of generations into a single DataFrame
total = [gen1df, gen2df, gen3df, gen4df]
totaldf = pd.concat(total)
totaldf = totaldf.reset_index()

Then, by isolating the types and its values, we can see the gamefreak devs preferences towards some pokemon types

In [13]:
#overall type DataFrame
total_types = totaldf['Types'].str[0]
total_types = total_types.value_counts()
types_df = pd.DataFrame(total_types).reset_index()
types_df = types_df.astype({'Types':'str', 'count': 'int'})

In [14]:
#overall types plot
type_colors = { 'water':'#6890F0','normal':'#A8A878','bug':'#A8B820','grass':'#78C850','fire':'#F08030','electric':'#F8D030','psychic':'#F85888','poison':'#A040A0','rock':'#B8A038','ground':'#E0C068','fighting':'#C03028','fairy':'#EE99AC','ice':'#98D8D8','dark':'#705848','ghost':'#705898','dragon':'#7038F8','steel':'#B8B8D0'}
type_plot = px.pie(types_df, names='Types', values='count',color='Types',color_discrete_map=type_colors)
type_plot.show()

During the early generations of pokemon, there was a preference of the devs to opt for water pokemons and normal ones

Doing the same with the pokemon attributes, we measure the strongest attribute in every pokemon

In [15]:
#overall attributes DataFrame
total_attributes = totaldf['Attribute']
total_attributes = total_attributes.value_counts()
attributes_df = pd.DataFrame(total_attributes).reset_index()

In [16]:
#overall attributes plot
attribute_colors= {'attack':'#DC143C','special-attack':'#6495ED','speed':'#E9967A','defense':'#FFD700','hp':'#B22222','special-defense':'#8FBC8F'}
attribute_plot = px.bar(attributes_df, x = 'Attribute', y = 'count', color='Attribute',color_discrete_map = attribute_colors)
attribute_plot.show()

Looking at the graphs we can see that pokemon in the early generations, used to have more physical attacks than special ones